In [ ]:
!pip install fastai --upgrade

     |████████████████████████████████| 194kB 8.6MB/s 
     |████████████████████████████████| 61kB 9.9MB/s 
     |████████████████████████████████| 12.8MB 220kB/s 
     |████████████████████████████████| 776.8MB 24kB/s 
ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.7.1 which is incompatible.
  Found existing installation: torch 1.8.1+cu101
    Uninstalling torch-1.8.1+cu101:
      Successfully uninstalled torch-1.8.1+cu101
  Found existing installation: torchvision 0.9.1+cu101
    Uninstalling torchvision-0.9.1+cu101:
      Successfully uninstalled torchvision-0.9.1+cu101
  Found existing installation: fastai 1.0.61
    Uninstalling fastai-1.0.61:
      Successfully uninstalled fastai-1.0.61


In [ ]:
import os
import glob
import time
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from tqdm.notebook import tqdm
from skimage.color import rgb2lab, lab2rgb

import torch
from torch import nn, optim
from torchvision import transforms
from torchvision.models.resnet import resnet18
from torch.utils.data import Dataset, DataLoader

from fastai.vision.learner import create_body
from fastai.vision.models.unet import DynamicUnet
from fastai.data.external import untar_data, URLs

In [ ]:
SIZE = 256

In [ ]:
class AverageMeter:
  def __init__(self):
    self.reset()

  def reset(self):
    self.count, self.avg, self.sum = 0., 0., 0.

  def update(self, val, count):
    self.count += count
    self.sum += val * count
    self.avg = self.sum / self.count

def init_loss_meters():
  loss_D_fake = AverageMeter()
  loss_D_real = AverageMeter()
  loss_D = AverageMeter()

  loss_G_GAN = AverageMeter()
  loss_G_L1 = AverageMeter()
  loss_G = AverageMeter()

  return {"loss_D_fake": loss_D_fake,
          "loss_D_real": loss_D_real,
          "loss_D": loss_D,
          "loss_G_GAN": loss_G_GAN,
          "loss_G_L1": loss_G_L1,
          "loss_G": loss_G}

def update_losses(model, loss_meters, count):
  for loss_name, loss_meter in loss_meters.items():
    loss = getattr(model, loss_name)
    loss_meter.update(loss.item(), count)

In [ ]:
def build_generator(n_inputs, n_outputs, core):
  body = create_body(core, pretrained=True, n_in=n_inputs, cut=-2)

  return DynamicUnet(body, n_outputs, (SIZE, SIZE))

In [ ]:
def train_generator(device, G, train_dl, val_dl, opt, criterion, epochs):
  for e in range(epochs):
    train_loss_meter = AverageMeter()
    val_loss_meter = AverageMeter()

    G.train()
    for data in tqdm(train_dl):
      L, ab = data["L"].to(device), data["ab"].to(device)

      preds = G(L)

      loss = criterion(preds, ab)

      opt.zero_grad()
      loss.backward()
      opt.step()

      train_loss_meter.update(loss.item(), L.size(0))

    G.eval()
    for data in tqdm(val_dl):
      L, ab = data["L"].to(device), data["ab"].to(device)

      preds = G(L)

      loss = criterion(preds, ab)

      val_loss_meter.update(loss.item(), L.size(0))

    print(f"Epoch {e + 1}/{epochs}")
    print(f"L1 --- Trn_loss: {train_loss_meter.avg:.4f} --- Val_loss: {val_loss_meter.avg:.4f}")
    torch.save(G.state_dict(), f"./gen_models/{e}_{time.time()}_res18-unet.pt")


In [ ]:
class TrainingDataset(Dataset):
  def __init__(self, paths):
    self.transforms = transforms.Compose([
            transforms.Resize((SIZE, SIZE)),
            transforms.RandomHorizontalFlip(),
        ])
    self.paths = paths

  def __getitem__(self, idx):
    img = Image.open(self.paths[idx]).convert("RGB")
    img = self.transforms(img)
    img = np.array(img)

    lab_img = rgb2lab(img).astype("float32")
    lab_img = transforms.ToTensor()(lab_img)

    L = lab_img[[0], ...] / 50. - 1.
    ab = lab_img[[1, 2], ...] / 110.

    return {"L": L, "ab": ab}

  def __len__(self):
    return len(self.paths)

class ValidationDataset(Dataset):
  def __init__(self, paths):
    self.transforms = transforms.Compose([
            transforms.Resize((SIZE, SIZE)),
        ])
    self.paths = paths

  def __getitem__(self, idx):
    img = Image.open(self.paths[idx]).convert("RGB")
    img = self.transforms(img)
    img = np.array(img)

    lab_img = rgb2lab(img).astype("float32")
    lab_img = transforms.ToTensor()(lab_img)

    L = lab_img[[0], ...] / 50. - 1.
    ab = lab_img[[1, 2], ...] / 110.

    return {"L": L, "ab": ab}

  def __len__(self):
    return len(self.paths)

In [ ]:
root = str(untar_data(URLs.COCO_SAMPLE)) + "/train_sample"

paths = glob.glob(root + "/*.jpg")

np.random.seed(42)
paths_subset = np.random.choice(paths, 12_000, replace=False)

rand_idxs = np.random.permutation(12_000)
train_idxs = rand_idxs[:10_000]
val_idxs = rand_idxs[10_000:]

train_paths = paths_subset[train_idxs]
val_paths = paths_subset[val_idxs]

train_dset = TrainingDataset(train_paths)
val_dset = ValidationDataset(val_paths)

train_dl = DataLoader(train_dset, 16, num_workers=2, pin_memory=True)
val_dl = DataLoader(val_dset, 8, num_workers=2, pin_memory=True)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
G = build_generator(1, 2, resnet18).to(device)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth


In [ ]:
l1_opt = optim.Adam(G.parameters(), lr=1e-4)
l1_loss = nn.L1Loss()

In [ ]:
train_generator(device, G, train_dl, val_dl, l1_opt, l1_loss, 30)


Epoch 1/30
L1 --- Trn_loss: 0.0870 --- Val_loss: 0.0803



Epoch 2/30
L1 --- Trn_loss: 0.0794 --- Val_loss: 0.0788



Epoch 3/30
L1 --- Trn_loss: 0.0766 --- Val_loss: 0.0798



Epoch 4/30
L1 --- Trn_loss: 0.0741 --- Val_loss: 0.0787



Epoch 5/30
L1 --- Trn_loss: 0.0715 --- Val_loss: 0.0797


...


Epoch 26/30
L1 --- Trn_loss: 0.0501 --- Val_loss: 0.0780



Epoch 27/30
L1 --- Trn_loss: 0.0495 --- Val_loss: 0.0785



Epoch 28/30
L1 --- Trn_loss: 0.0489 --- Val_loss: 0.0790



Epoch 29/30
L1 --- Trn_loss: 0.0484 --- Val_loss: 0.0787



Epoch 30/30
L1 --- Trn_loss: 0.0476 --- Val_loss: 0.0786


In [ ]:
!zip -r ./gen_models.zip ./gen_models

  adding: gen_models/ (stored 0%)
  adding: gen_models/17_1619678546.44514_res18-unet.pt (deflated 7%)
  adding: gen_models/4_1619672929.0411165_res18-unet.pt (deflated 7%)
  adding: gen_models/28_1619683290.488112_res18-unet.pt (deflated 7%)
  adding: gen_models/22_1619680706.0349762_res18-unet.pt (deflated 7%)
  adding: gen_models/16_1619678115.397143_res18-unet.pt (deflated 7%)
  adding: gen_models/24_1619681568.6929681_res18-unet.pt (deflated 7%)
  adding: gen_models/0_1619671187.9509108_res18-unet.pt (deflated 7%)
  adding: gen_models/21_1619680274.38992_res18-unet.pt (deflated 7%)
  adding: gen_models/7_1619674230.4844167_res18-unet.pt (deflated 7%)
  adding: gen_models/20_1619679842.199287_res18-unet.pt (deflated 7%)
  adding: gen_models/11_1619675958.1446617_res18-unet.pt (deflated 7%)
  adding: gen_models/19_1619679410.9194286_res18-unet.pt (deflated 7%)
  adding: gen_models/23_1619681137.5498242_res18-unet.pt (deflated 7%)
  adding: gen_models/9_1619675094.6967552_res18-unet.

In [ ]:
from google.colab import drive
drive.mount('./drive', force_remount=True)

Mounted at ./drive
